# Imports

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import gensim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import datasets, svm, tree, metrics
from sklearn.cluster import KMeans
from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas import DataFrame
from pathlib import Path
import collections
import numpy as np
import subprocess
import itertools
import os.path
import random
import time
import json
import sys
import re

# Load the Data

In [ ]:
with open('input_data.json') as data:
    jdata = json.load(data)

# Clean the Data

In [144]:
def codeToList(code):
    clean_string = ' '.join(code.split())
    the_list = re.split(' |\r\n',clean_string)
    return the_list

#from gensim.parsing.preprocessing import preprocess_string
#from gensim.parsing.preprocessing import strip_punctuation
#from gensim.parsing.preprocessing import strip_multiple_whitespaces

#def codeToList(code):
    
#    CUSTOM_FILTERS = [lambda x: x.lower(), strip_punctuation, strip_multiple_whitespaces]
#    the_list = preprocess_string(code, CUSTOM_FILTERS)
#    return the_list

# Function to Translate Operators to Natural Language

In [145]:
def naive_word_conversion(code_string):
    
    code_string = re.sub('\>\>=', ' is reassigned after shifting right by ', code_string)
    code_string = re.sub('\<\<=', ' is reassigned after shifting left by ', code_string)
    code_string = re.sub('\<\<', ' is shifted left by ', code_string)
    code_string = re.sub('\>\>', ' is shifted right by ' , code_string)
    code_string = re.sub('\<=', ' is less than or equal to ', code_string)
    code_string = re.sub('\>=', ' is greater than or equal to ', code_string)
    code_string = re.sub('-\>', ' is dereferenced and retrieves ', code_string)
    code_string = re.sub('\<', ' is less than ', code_string)
    code_string = re.sub('\>', ' is greater than ', code_string)
    code_string = re.sub('\+\+' ,' is incremented by one ' , code_string)
    code_string = re.sub('--', ' is decremented by one ', code_string)
    code_string = re.sub('\+=', ' is reassinged after adding ' , code_string)
    code_string = re.sub('\-=', ' is reassigned after subtracting ', code_string)
    code_string = re.sub('\*=', ' is reassigned after multiplying by ', code_string)
    code_string = re.sub('\/=', ' is reassigned after dividing by ', code_string)
    code_string = re.sub('\%=', ' is reassigned after moding by ', code_string)
    code_string = re.sub('&=', ' is reassigned after a bitwise AND of ', code_string)
    code_string = re.sub('\|=', ' is reassigned after a bitwise inclusive OR of ', code_string)
    code_string = re.sub('\^=', ' is reassigned after a bitwise exclusive OR of ', code_string)
    code_string = re.sub('==', ' is equal to ', code_string)
    code_string = re.sub('\!=', ' is not equal to ', code_string)
    code_string = re.sub('&&', ' and ', code_string)
    code_string = re.sub('\|\|', ' or ', code_string)
    code_string = re.sub('\?', ' is true then ', code_string)
    code_string = re.sub('\:', ' else is false then ', code_string)
    code_string = re.sub('\/\/', '  ', code_string)
    code_string = re.sub('=', ' equals ', code_string)
    code_string = re.sub('\.', ' uses ', code_string)
    code_string = re.sub(' -{1} ', ' is subtracted from ', code_string)
    code_string = re.sub('\+{1}', ' is added to ', code_string)
    code_string = re.sub('\/\*([^\*]|(\*+([^\*\/])))*\*\/', ' ', code_string)  
    code_string = re.sub('\*{1}', ' is multiplied by ', code_string)
    code_string = re.sub('&{1}', ' has an AND done with ', code_string)
    code_string = re.sub('\/{1}', ' is divided by ', code_string)
    code_string = re.sub('\%{1}', ' is moded by ', code_string)
    code_string = re.sub('\!{1}', ' is negated ', code_string)
    code_string = re.sub('\|{1}', ' has an OR done with ', code_string)
    code_string = re.sub('\^{1}', ' has an X OR done with ', code_string)
    code_string = re.sub('\(', ' open parenthesis ', code_string)
    code_string = re.sub('\)', ' close parenthesis ', code_string)
    code_string = re.sub('\[', ' open bracket ', code_string)
    code_string = re.sub('\]', ' close bracket ', code_string)
    code_string = re.sub('\#', ' ', code_string)
    code_string = re.sub(';', ' . ', code_string)
    
    return code_string

In [ ]:
if (False):
    def naive_word_conversion(code_string):

        code_string = re.sub('\>\>=', ' is reassigned after shifting right by ', code_string)
        code_string = re.sub('\<\<=', ' is reassigned after shifting left by ', code_string)
        code_string = re.sub('\<\<', ' is shifted left by ', code_string)
        code_string = re.sub('\>\>', ' is shifted right by ' , code_string)
        code_string = re.sub('\<=', ' is less than or equal to ', code_string)
        code_string = re.sub('\>=', ' is greater than or equal to ', code_string)
        code_string = re.sub('-\>', ' is dereferenced and retrieves ', code_string)
        code_string = re.sub('\<', ' is less than ', code_string)
        code_string = re.sub('\>', ' is greater than ', code_string)
        code_string = re.sub('\+\+' ,' is incremented by one ' , code_string)
        code_string = re.sub('--', ' is decremented by one ', code_string)
        code_string = re.sub('\+=', ' is reassinged after adding ' , code_string)
        code_string = re.sub('\-=', ' is reassigned after subtracting ', code_string)
        code_string = re.sub('\*=', ' is reassigned after multiplying by ', code_string)
        code_string = re.sub('\/=', ' is reassigned after dividing by ', code_string)
        code_string = re.sub('\%=', ' is reassigned after moding by ', code_string)
        code_string = re.sub('&=', ' is reassigned after a bitwise AND of ', code_string)
        code_string = re.sub('\|=', ' is reassigned after a bitwise inclusive OR of ', code_string)
        code_string = re.sub('\^=', ' is reassigned after a bitwise exclusive OR of ', code_string)
        code_string = re.sub('==', ' is equal to ', code_string)
        code_string = re.sub('\!=', ' is not equal to ', code_string)
        code_string = re.sub('&&', ' and ', code_string)
        code_string = re.sub('\|\|', ' or ', code_string)
        code_string = re.sub('\?', ' is true then ', code_string)
        code_string = re.sub('\:', ' else is false then ', code_string)
        code_string = re.sub('\/\/', '  ', code_string)
        code_string = re.sub('=', ' equals ', code_string)
        code_string = re.sub('\.', ' uses ', code_string)
        code_string = re.sub(' -{1} ', ' is subtracted from ', code_string)
        code_string = re.sub('\+{1}', ' is added to ', code_string)
        code_string = re.sub('\/\*([^\*]|(\*+([^\*\/])))*\*\/', ' ', code_string)  
        code_string = re.sub('\*{1}', ' is multiplied by ', code_string)
        code_string = re.sub('&{1}', ' has an AND done with ', code_string)
        code_string = re.sub('\/{1}', ' is divided by ', code_string)
        code_string = re.sub('\%{1}', ' is moded by ', code_string)
        code_string = re.sub('\!{1}', ' is negated ', code_string)
        code_string = re.sub('\|{1}', ' has an OR done with ', code_string)
        code_string = re.sub('\^{1}', ' has an X OR done with ', code_string)
        code_string = re.sub('\{', ' open curly brackets ', code_string)
        code_string = re.sub('\}', ' close curly brackets ', code_string)
        code_string = re.sub('\(', ' open parenthesis ', code_string)
        code_string = re.sub('\)', ' close parenthesis ', code_string)
        code_string = re.sub('\[', ' open bracket ', code_string)
        code_string = re.sub('\]', ' close bracket ', code_string)
        code_string = re.sub('\#', ' ', code_string)
        code_string = re.sub(';', ' . ', code_string)
        code_string = re.sub('[0-9]+', ' a number ', code_string)
        code_string = re.sub(',', ' ', code_string)
        code_string = re.sub('\.', ' ', code_string)

        return code_string


# Extract the Vectors 

In [146]:

samples = []
labels = []
index = []

for problem in jdata:
    for submissions in problem['Submissions']:
        for submission in submissions:
            code = submissions[submission]['Code'][0]
            code = naive_word_conversion(code)

            
            samples.append(codeToList(code))
            labels.append(str(problem['contestId']) + problem['index'])
            
            index.append(submission)

            
print(samples[0:10])

[['include', 'is', 'less', 'than', 'stdio', 'uses', 'h', 'is', 'greater', 'than', 'include', 'is', 'less', 'than', 'string', 'uses', 'h', 'is', 'greater', 'than', 'int', 'len', '.', 'char', 'data', 'open', 'bracket', '200', 'close', 'bracket', '.', 'int', 'main', 'open', 'parenthesis', 'close', 'parenthesis', '{', 'int', 'i,', 'cnt', 'equals', '0', '.', 'scanf', 'open', 'parenthesis', '"', 'is', 'moded', 'by', 's",', 'data', 'close', 'parenthesis', '.', 'len', 'equals', 'strlen', 'open', 'parenthesis', 'data', 'close', 'parenthesis', '.', 'for', 'open', 'parenthesis', 'i', 'equals', '0', '.', 'i', 'is', 'less', 'than', 'len', '.', 'i', 'is', 'incremented', 'by', 'one', 'close', 'parenthesis', '{', 'if', 'open', 'parenthesis', 'data', 'open', 'bracket', 'i', 'close', 'bracket', 'is', 'equal', 'to', "'o'", 'close', 'parenthesis', 'cnt', 'is', 'incremented', 'by', 'one', '.', '}', 'if', 'open', 'parenthesis', 'cnt', 'is', 'equal', 'to', '0', 'close', 'parenthesis', 'printf', 'open', 'pare

# Load a pre-trained Model

In [119]:
%%time
import gensim.downloader as api
# text8: First 100,000,000 bytes of plain text from Wikipedia
corpus = api.load('text8')  # download the corpus and return it opened as an iterable
corpus = [s for s in corpus]

CPU times: user 1.47 s, sys: 261 ms, total: 1.73 s
Wall time: 1.77 s


In [147]:
model = Word2Vec(size=300, window=4, min_count=1, workers=4, iter=1)  # train a model from the corpus
model.build_vocab(corpus + samples)

In [148]:
%%time
for epoch in range(2):
    if epoch % 2 == 0:
        print('Training epoch %s' %epoch)
    model.train(corpus,total_examples=model.corpus_count, epochs=model.iter)

Training epoch 0
CPU times: user 1min 48s, sys: 264 ms, total: 1min 49s
Wall time: 27.8 s


In [153]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.6506679058074951),
 ('elizabeth', 0.644037127494812),
 ('prince', 0.6368358731269836),
 ('daughter', 0.6255747079849243),
 ('mary', 0.6230384111404419),
 ('throne', 0.6156575679779053),
 ('emperor', 0.6111117601394653),
 ('judah', 0.6030086278915405),
 ('empress', 0.6020327806472778),
 ('son', 0.5919702053070068)]

# Train the NLP Model

In [150]:
for epoch in range(100):
    if epoch % 2 == 0:
        print('Training epoch %s' %epoch)
    model.train(samples,total_examples=model.corpus_count, epochs=model.iter)

Training epoch 0
Training epoch 2
Training epoch 4
Training epoch 6
Training epoch 8


# Create a List of  Samples

In [154]:
%%time 
vectorSamples = []
for sentence in samples:
    accumulatedVector = np.zeros(300)
    count =0
    for word in sentence:
        acumulatedVector = model.wv[word] + accumulatedVector  
        count=count+1
    
    vectorSamples.append(acumulatedVector/count)

    

CPU times: user 2.3 s, sys: 30.1 ms, total: 2.33 s
Wall time: 2.29 s


# Split the Data

In [155]:
samples_train, samples_test, labels_train, labels_test = train_test_split(
    vectorSamples, labels, test_size=0.33, random_state=42)


#labels_train
#samples_test
#labels_test
#samples_train
print("Length of Labels Train: "+str(len(labels_train)))
print("Length of Samples Train: "+str(len(samples_train)))
print()
print("Length of Labels TEST: "+str(len(labels_test)))
print("Length of Samples TEST: "+str(len(samples_test)))

Length of Labels Train: 2597
Length of Samples Train: 2597

Length of Labels TEST: 1280
Length of Samples TEST: 1280


# Create the Machine Learning Algorithms

In [156]:
svm_classifier = svm.SVC(kernel='linear',class_weight='balanced')


dt_classifier = tree.DecisionTreeClassifier(class_weight='balanced')


rf_classifier = RandomForestClassifier(n_estimators=10,class_weight='balanced')


nn_classifier = MLPClassifier(hidden_layer_sizes=(100,70),max_iter=500)



# Scoring with vectorSamples and labels

In [157]:
svm_scores = cross_val_score(svm_classifier, vectorSamples, labels, cv=5)
print('SVM scores: ' + str(svm_scores))
svm_average = sum(svm_scores)/5
print('SVM scores average: ' + str(svm_average))

dt_scores = cross_val_score(dt_classifier, vectorSamples, labels, cv=5)
print('DT scores:  ' + str(dt_scores))
dt_average = sum(dt_scores)/5
print('DT scores average: ' + str(dt_average))

rf_scores = cross_val_score(rf_classifier, vectorSamples, labels, cv=5)
print('RF scores:  ' + str(rf_scores))
rf_average = sum(rf_scores)/5
print('RF scores average: ' + str(rf_average))


nn_scores = cross_val_score(nn_classifier, vectorSamples, labels, cv=5)
print('NN scores:  ' + str(nn_scores))
nn_average = sum(nn_scores)/5
print('NN scores average: ' + str(nn_average))

SVM scores: [0.0754717  0.07535121 0.04241645 0.08246073 0.08322325]
SVM scores average: 0.07178466929833138
DT scores:  [0.14465409 0.13537676 0.13496144 0.14267016 0.12945839]
DT scores average: 0.13742416582972852
RF scores:  [0.13459119 0.12643678 0.14138817 0.14921466 0.14531044]
RF scores average: 0.13938824940042369
NN scores:  [0.18490566 0.17879949 0.1966581  0.19502618 0.17833554]
NN scores average: 0.18674499204502534


# Scoring with samples_test and labels_test

In [132]:
svm_scores = cross_val_score(svm_classifier, samples_test, labels_test, cv=5)
print('SVM scores: ' + str(svm_scores))
svm_average = sum(svm_scores)/5
print('SVM scores average: ' + str(svm_average))

dt_scores = cross_val_score(dt_classifier, samples_test, labels_test, cv=5)
print('DT scores:  ' + str(dt_scores))
dt_average = sum(dt_scores)/5
print('DT scores average: ' + str(dt_average))

rf_scores = cross_val_score(rf_classifier, samples_test, labels_test, cv=5)
print('RF scores:  ' + str(rf_scores))
rf_average = sum(rf_scores)/5
print('RF scores average: ' + str(rf_average))

nn_scores = cross_val_score(nn_classifier, samples_test, labels_test, cv=5)
print('NN scores:  ' + str(nn_scores))
nn_average = sum(nn_scores)/5
print('NN scores average: ' + str(nn_average))

/home/agostini/.conda/envs/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


SVM scores: [0.03623188 0.03007519 0.02702703 0.05691057 0.02575107]
SVM scores average: 0.03519914822439746


/home/agostini/.conda/envs/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


DT scores:  [0.0942029  0.11654135 0.0965251  0.13414634 0.09871245]
DT scores average: 0.10802562725492515


/home/agostini/.conda/envs/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


RF scores:  [0.07971014 0.11654135 0.11969112 0.12195122 0.10729614]
RF scores average: 0.1090379949706731


/home/agostini/.conda/envs/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


NN scores:  [0.14492754 0.16165414 0.14671815 0.14634146 0.17596567]
NN scores average: 0.15512138938781245


# Fit the Model

In [133]:
svm_classifier = svm.SVC(kernel='linear',class_weight='balanced')

#5-20 height for decision tree
dt_classifier = tree.DecisionTreeClassifier(class_weight='balanced')


rf_classifier = RandomForestClassifier(n_estimators=10,class_weight='balanced')


nn_classifier = MLPClassifier(hidden_layer_sizes=(100,70),max_iter=500)

svm_classifier.fit(samples_train,labels_train)
dt_classifier.fit(samples_train,labels_train)
rf_classifier.fit(samples_train,labels_train)
nn_classifier.fit(samples_train,labels_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 70), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

# Final Scores

In [134]:
score = svm_classifier.score(samples_test, labels_test)
print('SVM score: ' + str(score))
score = dt_classifier.score(samples_test, labels_test)
print('DT score: ' + str(score))
score = rf_classifier.score(samples_test, labels_test)
print('RF score: ' + str(score))
score = nn_classifier.score(samples_test, labels_test)
print('NN score: ' + str(score))

SVM score: 0.0546875
DT score: 0.13125
RF score: 0.140625
NN score: 0.17109375
